# Testing C++ code in python
June 12, 2020 \
June 20, 2020 : Added test for batch of images with multi-channels

In [46]:
import numpy as np

## Save image for C++ code

In [48]:
xsize,ysize=128,128
# xsize,ysize=10,10
batch_size,num_channels=200,2

In [49]:

### Save single 2D image to csv for C++ code to handle
fname='data/gen_images.npy'
a1=np.load(fname)
print(a1.shape)

a2=a1[0:(batch_size*num_channels),0,:xsize,:ysize]
print(a2.shape)
np.savetxt('data/images.csv',a2.flatten(),delimiter=',',newline=',')


(512, 1, 128, 128)
(400, 128, 128)


## Multi-channel batch example

### Read input

In [51]:
fname='data/images.csv'
x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(batch_size,num_channels,xsize,ysize)
print(x.shape)
# x

(200, 2, 128, 128)


In [52]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.


def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [53]:
def f_batch_histogram(img_arr,bins,norm,hist_range):
    ''' Compute histogram statistics for a batch of images'''

    ## Extracting the range. This is important to ensure that the different histograms are compared correctly
    if hist_range==None : ulim,llim=np.max(img_arr),np.min(img_arr)
    else: ulim,llim=hist_range[1],hist_range[0]
#         print(ulim,llim)
    ### array of histogram of each image
    hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important
    hist=np.stack(hist_arr[:,0]) # First element is histogram array
#         print(hist.shape)
    bin_list=np.stack(hist_arr[:,1]) # Second element is bin value 
    ### Compute statistics over histograms of individual images
    mean,err=np.mean(hist,axis=0),np.std(hist,axis=0)/np.sqrt(hist.shape[0])
    bin_edges=bin_list[0]
    centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    return mean,err,centers

In [54]:
bins=np.array([-0.5,0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5,8.5,10,20,30,40,50,60,70,80,90,100,300,500,700,900,1100,1300,1500,1700,1900,2100,2300]).astype(np.float64)
bins=f_transform(bins)
print(bins)


[-1.28571428 -0.77777778 -0.45454546 -0.23076923 -0.06666667  0.05882353
  0.15789474  0.23809524  0.30434782  0.36        0.42857143  0.66666667
  0.76470588  0.81818182  0.85185185  0.875       0.89189189  0.9047619
  0.91489362  0.92307692  0.97368421  0.98412698  0.98863636  0.99115044
  0.99275362  0.99386503  0.99468085  0.99530516  0.99579832  0.99619772
  0.99652778]


In [55]:
a=np.array([f_batch_histogram(x[:,channel,:,:],bins,False,None) for channel in range(num_channels)])
mean=a[:,0,:]
sdev=a[:,1,:]*np.sqrt(batch_size)

In [63]:
# mean,sdev

### Read from c++ files and compare

In [59]:
### Read c++ files
cp_ip_file='data/op_hist_mean.csv'
z_cpp1=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(num_channels,-1)

cp_ip_file='data/op_hist_sdev.csv'
z_cpp2=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(num_channels,-1)
# print(z_cpp1,'\n',z_cpp2)

In [60]:
### Check arrays
print(np.allclose(mean,z_cpp1,rtol=1e-4,atol=1e-8))

print(np.allclose(sdev,z_cpp2,rtol=1e-4,atol=1e-8))

True
True


## Conclusion:
###### July 23, 2020
Reading input + computing histogram mean and sdev \
Arrays from python and C++ match !!\
